In [1]:
import os
import torch
from d2l import torch as d2l

### 1. 下载和预处理数据集

In [2]:
data_dir = './fra-eng_dataset'


def read_data_nmt():
    """载入“英语－法语”数据集"""
    with open(os.path.join(data_dir, 'fra.txt'), 'r', encoding='utf-8') as f:
        return f.read()


raw_text = read_data_nmt()
print(raw_text[:75])

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



In [3]:
# 我们用空格代替不间断空格（non-breaking space）， 使用小写字母替换大写字母，并在单词和标点符号之间插入空格。
def preprocess_nmt(text):
    """预处理“英语－法语”数据集"""

    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char for i, char in enumerate(text)]
    return ''.join(out)


text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


### 2. 词元化

In [4]:
#@save
def tokenize_nmt(text, num_examples=None):
    """词元化“英语－法语”数据数据集"""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target


source, target = tokenize_nmt(text)
source[:6], target[:6]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!']])

### 3. 词汇表

In [5]:
# 构造词汇表，对于出现频率 < min_freg(2) 的次丢弃
src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

10012

### 4. 加载数据集 

In [6]:
# 设定相同的长度num_steps，如果文本序列的词元数目少于num_steps时，在其末尾添加特定的“<pad>”词元，直到长度达到num_steps
# 反之，截断文本序列，只取其前num_steps个词元，并且丢弃剩余的词元
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    if len(line) > num_steps:
        return line[:num_steps]  # 截断
    return line + [padding_token] * (num_steps - len(line))  # 填充


truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])
# 这里长度只有2，前两个对应标号[47, 4]，其他用标号[1]即'<pad>'填充

[47, 4, 1, 1, 1, 1, 1, 1, 1, 1]

In [7]:
def build_array_nmt(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成长度为num_steps的新句子"""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]  # <eos> end of sentence 标记句子结尾
    array = torch.tensor([truncate_pad(l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)  # 保存每句话实际有效的长度，排除填充的<pad>
    return array, valid_len

### 5. [封装]返回数据集的迭代器和词汇表

In [8]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词表"""
    # 1. 下载数据集并预处理(针对标点符号)
    text = preprocess_nmt(read_data_nmt())

    # 2. 词元化，返回原始数据和目标数据
    source, target = tokenize_nmt(text, num_examples)

    # 3. 词汇表
    src_vocab = d2l.Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = d2l.Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])

    # 4. 将机器翻译的文本序列转换成长度为num_steps的新句子
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)

    # 5. 构造数据迭代器
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = d2l.load_array(data_arrays, batch_size)

    return data_iter, src_vocab, tgt_vocab

### 6. 测试

In [9]:
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)
for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X.type(torch.int32))
    print('X的有效长度:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('Y的有效长度:', Y_valid_len)
    break

X: tensor([[ 87,  22,   4,   3,   1,   1,   1,   1],
        [  7, 101,   4,   3,   1,   1,   1,   1]], dtype=torch.int32)
X的有效长度: tensor([4, 4])
Y: tensor([[177, 178,  25,   4,   3,   1,   1,   1],
        [  6,   7, 158,   4,   3,   1,   1,   1]], dtype=torch.int32)
Y的有效长度: tensor([5, 5])


In [10]:
X.shape, X_valid_len.shape, Y.shape, Y_valid_len.shape

(torch.Size([2, 8]), torch.Size([2]), torch.Size([2, 8]), torch.Size([2]))

`注意到`
X: X是(2, 8), 2表示批量大小，即一次训练2个样本(句子)；8是时间步数，即句子长度

例如：
```
X = tensor(
[
    [ 87,  22,   4,   3,   1,   1,   1,   1],
    [  7, 101,   4,   3,   1,   1,   1,   1],
], dtype=torch.int32)
```
表示：第一句话的标号为[87, 22, 4, 3, 1, 1, 1, 1]，其中后面4个为<pad>补充，并非句子实际长度，所以

`X_valid_len = tensor([4, 4])`标记了第一句话有效长度只有 8 - 4 = 4